<img src="../images/logo.jpg" style="width:85px;height:85px;float:left" />
<h1 style="position:relative;float:left;display:inline">Writing a GAN using AshPy and TensorFlow Datasets</h1>

<a href='https://colab.research.google.com/github/zurutech/gans-from-theory-to-production/blob/master/3.%20AshPy/3.1.%20AshPy%20and%20TensorFlow%20Datasets.ipynb'>
    <img align="left" src='https://cdn-images-1.medium.com/max/800/1*ZpNn76K98snC9vDiIJ6Ldw.jpeg'></img>
</a>

In [1]:
! pip install --upgrade tensorflow-gpu==2.0.0beta1
# ! pip install --upgrade tensorflow==2.0.0beta1
! pip install --upgrade ashpy

Requirement already up-to-date: tensorflow-gpu==2.0.0beta1 in /data/pgaleone/env/lib/python3.7/site-packages (2.0.0b1)
Requirement already up-to-date: ashpy in /data/pgaleone/env/lib/python3.7/site-packages (1.0.2)


<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#GANs-with-AshPy-and-TensorFlow-Datasets-(tfds)" data-toc-modified-id="GANs-with-AshPy-and-TensorFlow-Datasets-(tfds)-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>GANs with AshPy and TensorFlow Datasets (tfds)</a></span><ul class="toc-item"><li><span><a href="#AshPy-Essentials" data-toc-modified-id="AshPy-Essentials-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>AshPy Essentials</a></span></li><li><span><a href="#tfds-and-AshPy-input-format" data-toc-modified-id="tfds-and-AshPy-input-format-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>tfds and AshPy input format</a></span><ul class="toc-item"><li><span><a href="#Getting-the-data-ready-to-use" data-toc-modified-id="Getting-the-data-ready-to-use-1.2.1"><span class="toc-item-num">1.2.1&nbsp;&nbsp;</span>Getting the data ready to use</a></span></li></ul></li></ul></li><li><span><a href="#DCGAN-Theory-and-Practice" data-toc-modified-id="DCGAN-Theory-and-Practice-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>DCGAN Theory and Practice</a></span><ul class="toc-item"><li><span><a href="#Generator:-from-noise-to-insight" data-toc-modified-id="Generator:-from-noise-to-insight-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Generator: from noise to insight</a></span><ul class="toc-item"><li><span><a href="#Deconvolution" data-toc-modified-id="Deconvolution-2.1.1"><span class="toc-item-num">2.1.1&nbsp;&nbsp;</span>Deconvolution</a></span></li><li><span><a href="#Batch-Normalization" data-toc-modified-id="Batch-Normalization-2.1.2"><span class="toc-item-num">2.1.2&nbsp;&nbsp;</span>Batch Normalization</a></span></li></ul></li><li><span><a href="#Discriminator" data-toc-modified-id="Discriminator-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Discriminator</a></span></li><li><span><a href="#Loss-function:-a-bridge-between-two-networks" data-toc-modified-id="Loss-function:-a-bridge-between-two-networks-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Loss function: a bridge between two networks</a></span></li><li><span><a href="#Training" data-toc-modified-id="Training-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Training</a></span></li><li><span><a href="#Tensorboard" data-toc-modified-id="Tensorboard-2.5"><span class="toc-item-num">2.5&nbsp;&nbsp;</span>Tensorboard</a></span></li></ul></li><li><span><a href="#Towards-Serving" data-toc-modified-id="Towards-Serving-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Towards Serving</a></span></li><li><span><a href="#Links" data-toc-modified-id="Links-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Links</a></span></li></ul></div>

## GANs with AshPy and TensorFlow Datasets (tfds)

In this notebook, we are going to try to define a GAN and its input data pipeline using the AshPy and `tfds`. Our aim is to build a **face generator** where each image is a $64\times64\times3$ tensor.

### AshPy Essentials

AshPy is a TensorFlow 2.0 library for (**distributed**) training, evaluation, model selection, and fast prototyping. It is designed to ease the burden of setting up all the nuances of the architectures built to train complex custom deep learning models.

The library has been designed following the TensorFlow 2.0 principles:

- Ease of use.
- Keras models centric.

(Some of ) the features AshPy offers are:

- You can distribute your model training using any `tf.distribute` distribution strategy without worrying about anything.
- You can develop a state of the art model with high-level intuitive code.
- It is fully compatible with the TensorFlow 2.0 ecosytem.
- Custom losses and training behaviours are easy to implement by defining custom `ashpy.Executor`.
- Fully compatibile with the Keras API specification and the `tf.keras` package.
- AshPy correctly uses `tf.function` to accelerate the training loop for you.
- Automatic Model selection and model saving-restore.
- Automatic integration with TensorBoard.
- Ready to use models and losses for fast prototyping.

![bella](images/dataset-ashpy.png)

**AshPy enforces a programming style.**

Every trainer accepts a Keras model (or more than one, it depends on the training type), a dataset, the executor (loss computation behavior) and the hyperparameters and all of them must implement well-defined interfaces.

In order to correctly separate the data input pipeline from the model, let's introduce `tensorflow-datasets` (`tfds`): the collection of datasets ready to use, all built upon the `tf.data.Dataset` API.

### tfds and AshPy input format

The `tf.data` API has been designed to write complex input pipelines in a very simple manner. It uses the **named pattern idiom** (also called **method chaining**) and its methods are inspired to the functional programming languages that applies transformations to lists.

The most imporant class is the `tf.data.Dataset` class that represents a sequence of elements: can apply transformation to this sequence of elements in order to create our dataset.

Every `ashpy.Trainer` requires a `tf.data.Dataset` object to use.

When using a `ashpy.AdversarialTrainer` that, as the name suggests, implements the adversarial training process, the `tf.data.Dataset` object should be in a well-known format.

In particular, the dataset return type must always be in the format

```python
tuple(tuple(a,b), c)
```

where:

- **a** is the input sample.
- **b** is the label/condition (if any, otherwise fill it with 0).
- **c** is the generator input, usually the __noise__.

Using `tensorflow-datasets` we don't have to bother about the download, preprocess and iterator generator: for the most common dataset everything is ready to use.

Downloading, having all the information, and a `tf.data.Dataset` object ready-to-use is just a single line.

#### Getting the data ready to use

In [2]:
import tensorflow as tf
import tensorflow_datasets as tfds
import ashpy
from IPython import display

dataset_name = "celeb_a"
if dataset_name not in tfds.list_builders():
    raise ValueError(f"Something wrong with tfds: missing {dataset_name}")
dataset, info = tfds.load(dataset_name,split=tfds.Split.ALL, with_info=True)
print(info.features["image"])

Image(shape=(218, 178, 3), dtype=tf.uint8)


Since our goal is to generate $64\times64$ images and training on batches, we can use method chaining on the returned `dataset` object to generate an optimized training input pipeline that fits all our needs.

Moreover, we have to follow the AshPy convention and return the data in the correct format:
```python
tuple(tuple(a,b), c)
```

In [3]:
def convert_and_resize(features):
    image = tf.image.convert_image_dtype(features["image"], dtype=tf.float32)
    image = (image - 0.5) * 2
    image = tf.image.resize(image, size=(64, 64))
    features["image"] = image
    return features


dataset = dataset.map(convert_and_resize)
# Now the dataset is a dataset of dictionaries
# setting it to the ashpy format.

latent_dimension = 100
dataset = dataset.map(lambda row: (
    (row["image"], 0), 
    tf.random.normal(shape=(latent_dimension,))
))

# Batch
batch_size = 32
dataset = dataset.batch(batch_size, drop_remainder=True).prefetch(1)

Downloading, post-processing and create an optimized input pipeline is just these few lines above!

Since we're working on images, we'll use an architecture created for this purpose: DCGAN <sup>[3](#3)</sup>.

## DCGAN Theory and Practice

![DCGAN](images/dcgan.png)

If you take a look at [this impressive list of GANs [2\]](#2), you would find out that **DCGAN**, the architecture of our choice, is a small drop in the ocean. While there may be sexier ones, very few offers the same level of clarity, performance and computational efficiency. For these reasons DCGAN is considered one of the cornerstones of this field.

Alec Radford, Luke Metz and Soumith Chintala proposed the architecture in [their 2015 paper [3\]](#3). The idea behind DCGAN is quite straightforward. Combining a set of architectural constraints with the power of CNN yielded a robust, stable and competitive model. Moreover, the architecture is simple: 4 deconvolutional layers for the `Generator` and 4 convolutional layers for the `Discriminator`. The constraints are the following:

- All pooling layers are replaced with strided convolutions (discriminator) and fractionally-strided convolutions (generator).
- Batch-normalization used in both networks.
- Removal of the fully-connected layers (except for the discriminator output).
- `ReLU` for all Generator layers except the output, which uses `tanh`.
- `LeakyReLU` activation in the discriminator for all layers.

Recently there have been some advancements in state of the art (e.g., Spectral Normalization). However, it is essential to have a firm understanding of the basic concepts. So, we leave you (an opinionated) list of further resources at the end of this notebook, to get you up to speed with the most exciting researches.

### Generator: from noise to insight

![DCGAN Generator](images/dcgan_generator.png)

Recalling the theoretical explanation, the Generator is the network responsible for the data-generation. It learns how to fool the discriminator, so it learns how to produce realistic results. Those results are then "sampled" from the learned manifold.

The most common type of generator input is noise (i.e.: random values). More specialized GANs may require extra parameters. Since our full-demo uses a Deep Convolutional GAN (DCGAN), we don't need any other parameters.

#### Deconvolution

Intuitively, the idea behind this operation is the following:

![Deconvolution](images/deconvolution.png)


> When neural networks generate images, they build the images starting from low-resolution high-level descriptions. In this way, the network starts describing a 
> rough representation and then fill in the details to create the final image.
>
>To do this, we need some way to go from a lower resolution image to a higher one. We generally do this with the deconvolution operation. 
>Roughly, deconvolution layers allow the model to use the points from the small image to “paint” a larger area in the bigger output image.

In practice, the deconvolution operation is often implemented by resizing, using bi-linear or nearest neighbor interpolation, followed by a convolution operation.

#### Batch Normalization

What you need to know about batch normalization is that is a layer that normalizes the values. TensorFlow makes it very easy to implement such operation:  

```python
tf.keras.layers.BatchNormalization()
```

The benefit of using batch normalization has been extensively discussed and proved in various papers. We do not enter into the theoretical depth of BatchNormalization, but you can refer to [[3\]](#3), [[8\]](#8) and [[9\]](#9) to learn more about the topic.

In [4]:
G = tf.keras.Sequential([
    tf.keras.layers.Dense(1024*4*4, use_bias=False, input_shape=(100,)),
    tf.keras.layers.Reshape((4,4,1024)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.LeakyReLU(),
    
    tf.keras.layers.Conv2DTranspose(256, (5, 5), strides=(2, 2), padding='same', use_bias=False),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.LeakyReLU(),

    tf.keras.layers.Conv2DTranspose(128, (5, 5), strides=(2, 2), padding='same', use_bias=False),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.LeakyReLU(),
    
    tf.keras.layers.Conv2DTranspose(64, (5, 5), strides=(2, 2), padding='same', use_bias=False),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.LeakyReLU(),
    
    tf.keras.layers.Conv2DTranspose(3, (5, 5), strides=(2, 2), padding='same', use_bias=False),
    tf.keras.layers.Activation(tf.math.tanh)
])

### Discriminator
![DCGAN Discriminator](images/dcgan_discriminator.png)

In [5]:
D = tf.keras.Sequential([
    tf.keras.layers.Conv2D(64, (5, 5), strides=(2, 2), padding='same', use_bias=False),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.LeakyReLU(),

    tf.keras.layers.Conv2D(128, (5, 5), strides=(2, 2), padding='same', use_bias=False),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.LeakyReLU(),
    
    tf.keras.layers.Conv2D(256, (5, 5), strides=(2, 2), padding='same', use_bias=False),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.LeakyReLU(),
    
    tf.keras.layers.Conv2D(512, (5, 5), strides=(2, 2), padding='same', use_bias=False),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.LeakyReLU(),
    
    tf.keras.layers.Reshape((-1,)),
    tf.keras.layers.Dense(1)
])

### Loss function: a bridge between two networks

There are a lot of loss functions usable in GANs' architectures. From very domain-specific ones to others that are perfect for general use cases. Since our goal here falls in the latter category, we use the so-called **Non-Saturating Loss** which is the non-saturating variant of the **MinMax Loss** proposed by Goodfellow in the [original paper [1\]](#1).

As stated above, one of `AshPy`'s beauties is its offer of ready-to-use losses. If you cannot find the loss you want, you can create your own. 

We use the following two losses:

```python
from ashpy.losses.gan import DiscriminatorMinMax, GeneratorBCE
generator_bce = GeneratorBCE()
minmax = DiscriminatorMinMax()
```

It is that easy.

In [6]:
from ashpy.losses.gan import DiscriminatorMinMax, GeneratorBCE
generator_bce = GeneratorBCE()
minmax = DiscriminatorMinMax()

### Training

Using AshPy the training loop is already implemented and everyting is ready to be used in distributed or local fashion, witout any change.

Just define the hyperparameters, like the optimizers to use, the metrics to measure, and the number of epochs to train, and the trainer object will do everything for you.

In [7]:
from ashpy.trainers.gan import AdversarialTrainer

epochs = 3
logdir = "log"
metrics = []

trainer = AdversarialTrainer(
    generator=G,
    discriminator=D,
    generator_optimizer=tf.optimizers.Adam(1e-4),
    discriminator_optimizer=tf.optimizers.Adam(1e-4),
    generator_loss=generator_bce,
    discriminator_loss=minmax,
    epochs=epochs,
    metrics=metrics,
    logdir=logdir,
)

trainer(dataset)

Initializing checkpoint.


W0731 16:59:18.078646 139803233097344 deprecation.py:323] From /data/pgaleone/env/lib/python3.7/site-packages/tensorflow/python/ops/nn_impl.py:182: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


[10] g_loss: 2.880147933959961 - d_loss: 0.13925009965896606
[20] g_loss: 3.069056987762451 - d_loss: 0.15181107819080353
[30] g_loss: 4.5184326171875 - d_loss: 0.07014752924442291
[40] g_loss: 2.5083045959472656 - d_loss: 0.19835466146469116
[50] g_loss: 4.271537780761719 - d_loss: 0.10161928832530975
[60] g_loss: 4.688693046569824 - d_loss: 0.012059198692440987
[70] g_loss: 5.356297016143799 - d_loss: 0.009372382424771786
[80] g_loss: 5.599665641784668 - d_loss: 0.03036346286535263
[90] g_loss: 3.650528907775879 - d_loss: 0.020713627338409424
[100] g_loss: 6.607163429260254 - d_loss: 0.1916131228208542
[110] g_loss: 5.402006149291992 - d_loss: 0.06360292434692383
[120] g_loss: 5.75997257232666 - d_loss: 0.007841583341360092
[130] g_loss: 5.959902286529541 - d_loss: 0.047625765204429626
[140] g_loss: 7.603246688842773 - d_loss: 0.016379475593566895
[150] g_loss: 5.817185401916504 - d_loss: 0.0031179115176200867
[160] g_loss: 4.176515579223633 - d_loss: 0.13304701447486877
[170] g_loss

[1320] g_loss: 6.8618855476379395 - d_loss: 0.1376517415046692
[1330] g_loss: 13.001863479614258 - d_loss: 0.17348669469356537
[1340] g_loss: 8.795181274414062 - d_loss: 0.02578847110271454
[1350] g_loss: 7.695051670074463 - d_loss: 0.010609918273985386
[1360] g_loss: 4.189368724822998 - d_loss: 0.06314592063426971
[1370] g_loss: 9.882402420043945 - d_loss: 0.023947272449731827
[1380] g_loss: 4.222321510314941 - d_loss: 0.028883960098028183
[1390] g_loss: 4.040860652923584 - d_loss: 0.037359997630119324
[1400] g_loss: 7.8968658447265625 - d_loss: 0.1010407954454422
[1410] g_loss: 11.822794914245605 - d_loss: 0.05045554041862488
[1420] g_loss: 3.7367658615112305 - d_loss: 0.14984391629695892
[1430] g_loss: 3.6628804206848145 - d_loss: 0.1618117243051529
[1440] g_loss: 6.9797868728637695 - d_loss: 0.014646289870142937
[1450] g_loss: 9.203269958496094 - d_loss: 0.10707372426986694
[1460] g_loss: 7.795417785644531 - d_loss: 0.09309379756450653
[1470] g_loss: 6.086577415466309 - d_loss: 0.0

[2610] g_loss: 4.506746292114258 - d_loss: 0.062271103262901306
[2620] g_loss: 4.281346321105957 - d_loss: 0.08114263415336609
[2630] g_loss: 5.045925140380859 - d_loss: 0.017104394733905792
[2640] g_loss: 4.617086410522461 - d_loss: 0.020673995837569237
[2650] g_loss: 5.275957107543945 - d_loss: 0.024622447788715363
[2660] g_loss: 5.276254653930664 - d_loss: 0.0685260146856308
[2670] g_loss: 9.315313339233398 - d_loss: 0.17118321359157562
[2680] g_loss: 7.673587799072266 - d_loss: 0.016747185960412025
[2690] g_loss: 3.5230712890625 - d_loss: 0.06679825484752655
[2700] g_loss: 8.62301254272461 - d_loss: 0.03311629593372345
[2710] g_loss: 6.30153751373291 - d_loss: 0.13613155484199524
[2720] g_loss: 3.5895211696624756 - d_loss: 0.1290828138589859
[2730] g_loss: 5.462795734405518 - d_loss: 0.045805253088474274
[2740] g_loss: 5.333235740661621 - d_loss: 0.23831063508987427
[2750] g_loss: 6.652834415435791 - d_loss: 0.04032079502940178
[2760] g_loss: 6.814387798309326 - d_loss: 0.012233143

...

[12900] g_loss: 7.088155746459961 - d_loss: 0.006402037106454372
[12910] g_loss: 3.9798784255981445 - d_loss: 0.02419377863407135
[12920] g_loss: 8.592244148254395 - d_loss: 0.0005976919201202691
[12930] g_loss: 4.7789716720581055 - d_loss: 0.01568216271698475
[12940] g_loss: 9.186971664428711 - d_loss: 0.0015794531209394336
[12950] g_loss: 4.872130393981934 - d_loss: 0.02511245757341385
[12960] g_loss: 12.577229499816895 - d_loss: 0.006449121981859207
[12970] g_loss: 9.434989929199219 - d_loss: 0.019527249038219452
[12980] g_loss: 7.569867134094238 - d_loss: 0.023286322131752968
[12990] g_loss: 8.0137939453125 - d_loss: 0.01214882917702198
[13000] g_loss: 7.407491207122803 - d_loss: 0.09360869228839874
[13010] g_loss: 9.039164543151855 - d_loss: 0.03027210384607315
[13020] g_loss: 7.854531764984131 - d_loss: 0.011589953675866127
[13030] g_loss: 8.065960884094238 - d_loss: 0.009656593203544617
[13040] g_loss: 5.012515544891357 - d_loss: 0.038367897272109985
[13050] g_loss: 6.9603271484

[14170] g_loss: 3.567824363708496 - d_loss: 0.13282448053359985
[14180] g_loss: 7.424173355102539 - d_loss: 0.0037338314577937126
[14190] g_loss: 10.335381507873535 - d_loss: 0.03227660059928894
[14200] g_loss: 6.368151664733887 - d_loss: 0.01651807501912117
[14210] g_loss: 5.09851598739624 - d_loss: 0.015047062188386917
[14220] g_loss: 8.822807312011719 - d_loss: 0.02754931151866913
[14230] g_loss: 13.79527473449707 - d_loss: 0.5821859240531921
[14240] g_loss: 8.049128532409668 - d_loss: 0.001115133287385106
[14250] g_loss: 6.891340255737305 - d_loss: 0.003551778383553028
[14260] g_loss: 7.878688812255859 - d_loss: 0.03215712308883667
[14270] g_loss: 5.100851535797119 - d_loss: 0.1225036084651947
[14280] g_loss: 7.427634239196777 - d_loss: 0.02123846486210823
[14290] g_loss: 7.4545488357543945 - d_loss: 0.01094632875174284
[14300] g_loss: 1.725752830505371 - d_loss: 0.3604505956172943
[14310] g_loss: 6.852899551391602 - d_loss: 0.018400290980935097
[14320] g_loss: 8.498245239257812 - 

[15440] g_loss: 9.764190673828125 - d_loss: 0.007662746123969555
[15450] g_loss: 10.002243041992188 - d_loss: 0.008847766555845737
[15460] g_loss: 5.415212154388428 - d_loss: 0.005451396107673645
[15470] g_loss: 7.238577842712402 - d_loss: 0.10436121374368668
[15480] g_loss: 11.229097366333008 - d_loss: 0.013464980758726597
[15490] g_loss: 7.534399032592773 - d_loss: 0.00880552176386118
[15500] g_loss: 8.816573143005371 - d_loss: 0.0046216947957873344
[15510] g_loss: 3.3204073905944824 - d_loss: 0.08592414110898972
[15520] g_loss: 6.426639556884766 - d_loss: 0.02250845357775688
[15530] g_loss: 3.0490031242370605 - d_loss: 0.15282416343688965
[15540] g_loss: 6.531001091003418 - d_loss: 0.012970378622412682
[15550] g_loss: 5.63796329498291 - d_loss: 0.017803829163312912
[15560] g_loss: 5.611639022827148 - d_loss: 0.03644729405641556
[15570] g_loss: 4.404098987579346 - d_loss: 0.019720956683158875
[15580] g_loss: 7.24697208404541 - d_loss: 0.0026997714303433895
[15590] g_loss: 7.320043563

[16710] g_loss: 10.075323104858398 - d_loss: 0.21856309473514557
[16720] g_loss: 7.846928119659424 - d_loss: 0.0017572209471836686
[16730] g_loss: 6.814908027648926 - d_loss: 0.015303613618016243
[16740] g_loss: 9.0933256149292 - d_loss: 0.0018623846117407084
[16750] g_loss: 7.203953742980957 - d_loss: 0.07151894271373749
[16760] g_loss: 6.572636127471924 - d_loss: 0.008813848719000816
[16770] g_loss: 8.500285148620605 - d_loss: 0.0026486944407224655
[16780] g_loss: 16.605613708496094 - d_loss: 0.216607004404068
[16790] g_loss: 9.259700775146484 - d_loss: 0.0006276571075432003
[16800] g_loss: 16.684886932373047 - d_loss: 0.013015763834118843
[16810] g_loss: 20.396869659423828 - d_loss: 0.2787688970565796
[16820] g_loss: 2.6278786659240723 - d_loss: 0.16116981208324432
[16830] g_loss: 5.178875923156738 - d_loss: 0.020776834338903427
[16840] g_loss: 14.474478721618652 - d_loss: 0.030531778931617737
[16850] g_loss: 10.746414184570312 - d_loss: 0.00016322944429703057
[16860] g_loss: 9.6786

[17980] g_loss: 7.606813430786133 - d_loss: 0.008291068486869335
[17990] g_loss: 8.336542129516602 - d_loss: 0.03601252660155296
[18000] g_loss: 7.180501461029053 - d_loss: 0.07423246651887894
[18010] g_loss: 5.092038154602051 - d_loss: 0.010299715213477612
[18020] g_loss: 6.502446174621582 - d_loss: 0.0033976470585912466
[18030] g_loss: 4.578167915344238 - d_loss: 0.02134871482849121
[18040] g_loss: 5.278720378875732 - d_loss: 0.026429854333400726
[18050] g_loss: 2.615992546081543 - d_loss: 0.09754433482885361
[18060] g_loss: 3.6511971950531006 - d_loss: 0.06073809787631035
[18070] g_loss: 13.947219848632812 - d_loss: 0.0005266036605462432
[18080] g_loss: 7.147188663482666 - d_loss: 0.01797708123922348
[18090] g_loss: 7.760547161102295 - d_loss: 0.007664849516004324
[18100] g_loss: 7.720438480377197 - d_loss: 0.013490918092429638
[18110] g_loss: 10.467683792114258 - d_loss: 0.050906017422676086
[18120] g_loss: 4.471699237823486 - d_loss: 0.06854616850614548
[18130] g_loss: 6.942570686

### Tensorboard

In [8]:
# Load the TensorBoard notebook extension and run it
%load_ext tensorboard
%tensorboard --logdir log

## Towards Serving

The path to the production is straightforward. We only have to save the model parameters and export them as a graph.
Begin our generator a Keras model, saving it to the disk is as easy as calling the `save_weights` method in the Generator model.

In [9]:
G.save_weights('generator')

---
## Links

<a id="1">[1]</a>: Generative Adversarial Networks : https://arxiv.org/abs/1406.2661

<a id="2">[2]</a>: really-awesome-gan : https://github.com/nightrome/really-awesome-gan

<a id="3">[3]</a>: DCGAN : https://arxiv.org/abs/1511.06434

<a id="4">[4]</a>: Batch Normalization: Accelerating Deep Network Training by Reducing Internal Covariate Shift : https://arxiv.org/abs/1502.03167

<a id="5">[5]</a>: How Does Batch Normalization Help Optimization? : https://arxiv.org/abs/1805.11604